In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as func
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType

spark = SparkSession.builder.appName("temperatura-max-min").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/14 14:03:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/10/14 14:03:41 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
schema = StructType([\
                StructField("stationID", StringType(),True),\
                StructField("date", IntegerType(), True),\
                StructField("measure_type", StringType(), True),\
                StructField("temperature", FloatType(), True)\
                ])

df = spark.read.schema(schema).csv("file:////home/vladimir/spark_udemy/temperature-data/1800.csv")


In [4]:
df.printSchema()


min_temps = df.filter(df.measure_type == 'TMIN')
station_temps = df.select("stationID","temperature")

min_temps_sta = min_temps.groupBy("stationID").min("temperature")
min_temps_sta.show()

root
 |-- stationID: string (nullable = true)
 |-- date: integer (nullable = true)
 |-- measure_type: string (nullable = true)
 |-- temperature: float (nullable = true)



+-----------+----------------+
|  stationID|min(temperature)|
+-----------+----------------+
|ITE00100554|          -148.0|
|EZE00100082|          -135.0|
+-----------+----------------+



In [6]:
min_temps_sta_F = min_temps_sta.withColumn("temperature",
                                           func.round(func.col("min(temperature)") * 0.1 * (9/5)
                                                      )).select("stationID","temperature").sort("temperature")
min_temps_sta_F.show()

+-----------+-----------+
|  stationID|temperature|
+-----------+-----------+
|ITE00100554|      -27.0|
|EZE00100082|      -24.0|
+-----------+-----------+

